In [16]:
import tensorflow as tf
from tensorflow import keras
import pathlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [27]:
img_shape = (128,128,3)
num_classes = 5

In [29]:
X=[]
y=[]
for path in pathlib.Path(r"data/data_bis/").iterdir():
    for path2 in pathlib.Path(path).iterdir():
        index = int(str(path).split("\\")[-1])-1
        image=np.array(plt.imread(path2))
        X.append(list(image))
        matrix=np.zeros((num_classes,))
        matrix[index] = 1
        y.append(matrix)

X=np.array(X)
y=np.array(y)
           
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)

In [30]:
base_model = keras.applications.resnet50.ResNet50(
    weights='imagenet',
    input_shape=img_shape,
    include_top=False)

base_model.trainable = False

In [31]:
inputs = keras.Input(shape=img_shape)
x = tf.cast(inputs, tf.float32)
x = tf.keras.applications.resnet50.preprocess_input(x)

In [32]:
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(num_classes)(x)
model = keras.Model(inputs, outputs)

In [33]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=keras.metrics.BinaryAccuracy())
hist = model.fit(X_train, y_train,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10
10/10 [==============================] - 10s 711ms/step - loss: 0.5741 - binary_accuracy: 0.7748 - val_loss: 0.4267 - val_binary_accuracy: 0.8000
Epoch 2/10
10/10 [==============================] - 6s 593ms/step - loss: 0.4195 - binary_accuracy: 0.8173 - val_loss: 0.2413 - val_binary_accuracy: 0.9000
Epoch 3/10
10/10 [==============================] - 6s 596ms/step - loss: 0.3159 - binary_accuracy: 0.8605 - val_loss: 0.2023 - val_binary_accuracy: 0.9000
Epoch 4/10
10/10 [==============================] - 6s 603ms/step - loss: 0.2569 - binary_accuracy: 0.8664 - val_loss: 0.1758 - val_binary_accuracy: 0.9000
Epoch 5/10
10/10 [==============================] - 6s 597ms/step - loss: 0.2136 - binary_accuracy: 0.9003 - val_loss: 0.1351 - val_binary_accuracy: 0.9500
Epoch 6/10
10/10 [==============================] - 6s 609ms/step - loss: 0.1835 - binary_accuracy: 0.9189 - val_loss: 0.1336 - val_binary_accuracy: 0.9500
Epoch 7/10
10/10 [==============================] - 7s 716ms/st

In [43]:
for k in np.round(model.predict(X_test)):
    print(list(k).index(max(k))+1)

1/1 [==============================] - 0s 134ms/step
3
3
2
5


In [40]:
y_test

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.]])